In [1]:
from telethon.sync import TelegramClient as TC
from telethon import TelegramClient
import telethon
import json
from PIL import Image
from io import BytesIO
import networkx as nx

Reading app_id and hash

In [2]:

with open("telegram_app_id.txt") as f:
    app_id = f.read().strip()
with open("telegram_api_hash.txt") as f:
    app_hash = f.read().strip()


Creating Telegram Client

In [3]:
client = TC("anon", app_id, app_hash)

Getting Myself

In [4]:
async with client:
    user = await client.get_me()
    print(user.username)

Pizoz


Getting Selected Group to analyse

In [5]:
id_Prodigeek = 1163643200

async with client:
    prodigeek_group = await client.get_entity(id_Prodigeek)
    print(type(prodigeek_group))
print(prodigeek_group.stringify())

<class 'telethon.tl.types.Channel'>
Channel(
	id=1163643200,
	title='Prodigeek Official',
	photo=ChatPhoto(
		photo_id=6023591964085171127,
		dc_id=4,
		has_video=False,
		stripped_thumb=b'\x01\x08\x08\xd5\xfd\xe7\x9aq\xf7}\xe8\xa2\x8a\x05c'
	),
	date=datetime.datetime(2025, 3, 14, 9, 38, 48, tzinfo=datetime.timezone.utc),
	creator=False,
	left=False,
	broadcast=False,
	verified=False,
	megagroup=True,
	restricted=False,
	signatures=False,
	min=False,
	scam=False,
	has_link=True,
	has_geo=False,
	slowmode_enabled=False,
	call_active=False,
	call_not_empty=False,
	fake=False,
	gigagroup=False,
	noforwards=False,
	join_to_send=True,
	join_request=False,
	forum=False,
	stories_hidden=False,
	stories_hidden_min=False,
	stories_unavailable=True,
	signature_profiles=False,
	access_hash=2525691116672421276,
	username=None,
	restriction_reason=[
	],
	admin_rights=None,
	banned_rights=None,
	default_banned_rights=ChatBannedRights(
		until_date=datetime.datetime(2038, 1, 19, 3, 14, 7, tzinfo=dat

Join Channel Request

In [6]:
from telethon.tl.functions.channels import JoinChannelRequest
async with client:
    await client(JoinChannelRequest(prodigeek_group.id))


Getting the Group Chat from client chats

In [7]:
async with client:
    dialogs = await client.get_dialogs()
for dialog in dialogs:
    if 'prodigeek official' == dialog.name.lower():
        prodigeek_dialog = dialog
        print(dialog.id)
        break

-1001163643200


Getting N messages from the Group Chat

In [8]:
group_messages = []

async with client:
    async for message in client.iter_messages(dialog, limit=3000):
        group_messages.append(message)


In [23]:
len(group_messages)


def createMessageDict(messages):
    # create a dictionary of messages
    d = dict()
    for m in messages:
        d[m.id] = m
    return d


def get_msg_author(msg_obj):
    # get the author of a message
    if msg_obj != None:
        if msg_obj.from_id and 'user_id' in msg_obj.from_id.__dict__:
            return msg_obj.from_id.user_id
        else:
            return None
    return None

def usersFromMessages(messages):
    # getting the users from the messages
    users = set()
    for m in messages:
        user_id = get_msg_author(m)
        if user_id != None:
            users.add(user_id)
    return users

messages_dict = createMessageDict(group_messages)
utenti = usersFromMessages(group_messages)

In [15]:
len(utenti), len(group_messages)

(267, 3000)

Identifying messages without replies

In [10]:

def noReplyMessages(messages):
    not_reply_message = []
    for m in messages:
        if m.reply_to == None:
            not_reply_message.append(m.id)
    return not_reply_message
no_reply_messages = noReplyMessages(group_messages)
len(no_reply_messages)

856

Identify interactions between users who reply to each other and mapping them on a graph

In [ ]:

def getRepliesList(messages):
    replies_list = []
    for m in messages:
        # if the message is a reply to another message (even if it's not in the N messages) saving the message pair
        if m.reply_to:
            m_a = m.reply_to.reply_to_msg_id
            m_b = m.id
            replies_list.append((m_a, m_b))
    return replies_list

def getInteractionsGraph(replies_list):
    G = nx.DiGraph()
    for m_a, m_b in replies_list:
        # getting the messages from the ids using the dictionary
        msg_a = messages_dict.get(m_a)
        msg_b = messages_dict.get(m_b)
        # getting the authors of the messages
        A, B = get_msg_author(msg_a), get_msg_author(msg_b)
        if A != None and B != None:
            # adding the edge to the graph
            G.add_edge(B,A)
    return G

replies_list = getRepliesList(group_messages)
msg_graph = getInteractionsGraph(replies_list)
msg_graph.order(), msg_graph.size()
nx.write_gexf(msg_graph, "Prodigeek.gexf")


For each link between two users we provide the list of replies exchanged between them 

In [12]:
def addLinkInfo(G, messages_dict, replies_list):
    for m_a, m_b in replies_list:
        msg_a = messages_dict.get(m_a)
        msg_b = messages_dict.get(m_b)
        A, B = get_msg_author(msg_a), get_msg_author(msg_b)
        if A is not None and B is not None and G.has_node(A) and G.has_node(B):
            if 'messaggi' in G[B][A]:
                G[B][A]['messaggi'].append((msg_b.message, msg_b.date))
            else:
                G[B][A]['messaggi'] = [(msg_b.message, msg_b.date)]
    return G
        
msg_graph = addLinkInfo(msg_graph, messages_dict, replies_list)

In [ ]:
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch, ChannelParticipantCreator

async def getRoles(group_id):
    roles = dict()
    async with client:
        participants = await client(GetParticipantsRequest(group_id, ChannelParticipantsSearch(''),0,50,hash=0))
    
    for p in participants.participants:
        if type(p) == ChannelParticipantCreator:
            roles[p.user_id] = "Owner"
            continue
        elif p.rank == None:
            roles[p.user_id] = "Admin"
            continue
        roles[p.user_id] = p.rank
    return roles
roles = await getRoles(id_Prodigeek)

        
def addUserRoles(G, roles):
    for node in G.nodes():
        if node in roles:
            G.nodes[node]['role'] = roles[node]
        else:
            G.nodes[node]['role'] = "User"
    return G
        
msg_graph = addUserRoles(msg_graph, roles)


In [26]:

for n, info in msg_graph.nodes(data=True):
    print(n, info)

679488937 {'role': 'User'}
796596281 {'role': 'User'}
1540263137 {'role': 'User'}
877555055 {'role': 'User'}
545125709 {'role': 'User'}
202830005 {'role': 'User'}
147414742 {'role': 'User'}
155278659 {'role': 'User'}
1220749864 {'role': 'User'}
7593210604 {'role': 'User'}
144635900 {'role': 'User'}
1109369959 {'role': 'User'}
124421473 {'role': 'User'}
626515711 {'role': 'User'}
230869637 {'role': 'User'}
6710129119 {'role': 'User'}
927177782 {'role': 'User'}
5788659229 {'role': 'User'}
572151306 {'role': 'Moderatore'}
539979445 {'role': 'User'}
203797998 {'role': 'User'}
288813584 {'role': 'User'}
6457517907 {'role': 'User'}
700862436 {'role': 'User'}
1306025285 {'role': 'User'}
193365423 {'role': 'User'}
6161488800 {'role': 'User'}
978092428 {'role': 'User'}
1587460589 {'role': 'User'}
86725266 {'role': 'User'}
5965381639 {'role': 'User'}
321622456 {'role': 'User'}
71091322 {'role': 'User'}
7000048094 {'role': 'User'}
157902776 {'role': 'User'}
1476482734 {'role': 'User'}
132520499 {

In [21]:
dizionario = dict()

if dizionario.get("ciao"):
    print("ciao")
else:
    print("fanculo")

fanculo
